<a href="https://colab.research.google.com/github/ThiaguinioB/Classic2Quantum/blob/main/Classic2Quantum_Biancucci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Classic2Quantum**

*Classic2Quantum es una herramienta que toma como entrada una tabla de verdad, calcula la ecuación lógica booleana correspondiente, la minimiza y la convierte en una ecuación cuántica. A partir de ella genera el circuito cuántico equivalente utilizando compuertas reversibles.*

# Ejecución de Classic2Quantum cargando una tabla de verdad de dos a cuatro variables por consola.

In [ ]:
# -*- coding: utf-8 -*-
"""
Classic2Quantum: Traductor de lógica booleana clásica a lógica cuántica.
Autor: Biancucci, Thiago
Versión: 1.0.0
"""

# =============================================================================
# Instalaciones (para notebook/Colab)
# =============================================================================
!pip install -q qiskit qiskit-aer
!pip install -q pyeda==0.28.0

# =============================================================================
# Imports
# =============================================================================
import os, platform, sys, collections, math
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import AerSimulator

import pyeda
from pyeda.inter import exprvars, truthtable, truthtable2expr, espresso_exprs
# Fix para Python 3.10+ (compatibilidad pyeda 0.28)
import collections.abc
collections.Sequence = collections.abc.Sequence


# =============================================================================
# Utilidades
# =============================================================================
def clear_console():
    """Limpia consola o salida embebida."""
    if 'ipykernel' in sys.modules:  # Jupyter / Colab
        from IPython.display import clear_output
        clear_output(wait=True)
    else:
        os.system('cls' if platform.system() == 'Windows' else 'clear')

clear_console()

print("PyEDA:", pyeda.__version__)
print("Qiskit:", qiskit.__version__)
print("Python:", sys.version.split()[0])


# ---------- helpers de parser ----------
def _clean(s: str) -> str:
    return s.replace(' ', '')

def _normalize_var(token: str) -> str:
    # 'x[0]' -> 'x0'
    if token.startswith('x[') and token.endswith(']'):
        return 'x' + token[2:-1]
    return token

def _split_top_level(csv: str) -> list:
    """Separa por comas a nivel de paréntesis 0 (no rompe subexpresiones)."""
    items, depth, start = [], 0, 0
    for i, ch in enumerate(csv):
        if ch == '(':
            depth += 1
        elif ch == ')':
            depth -= 1
        elif ch == ',' and depth == 0:
            items.append(csv[start:i])
            start = i + 1
    items.append(csv[start:])
    return items

def _unwrap(op: str, s: str) -> str:
    """
    Devuelve el contenido dentro de op(...).
    Ej: _unwrap('Or', 'Or(A,B)') -> 'A,B'
    """
    s = _clean(s)
    assert s.startswith(op + '(') and s.endswith(')'), f"Formato inesperado: {s}"
    return s[len(op) + 1 : -1]   # <- correcto (antes se comía letras)


def parse_expr_to_sop(expr) -> list:
    """
    Devuelve SOP como lista de términos:
      [[('x0', True), ('x1', False)], [('x0', False)], ...]
    Soporta: Or(...), And(...), literales (~x[1], x0,...)
    """
    s = _clean(str(expr))

    if s == '0':
        return []      # y ^= 0
    if s == '1':
        return [[]]    # y ^= 1

    # Or(...)
    if s.startswith('Or(') and s.endswith(')'):
        inner = _unwrap('Or', s)
        parts = _split_top_level(inner)
        terms = []
        for p in parts:
            terms.extend(parse_expr_to_sop(p))
        return terms

    # And(...)
    if s.startswith('And(') and s.endswith(')'):
        inner = _unwrap('And', s)
        lits = _split_top_level(inner)
        term = []
        for lit in lits:
            lit = _clean(lit)
            neg = lit.startswith('~')
            v = _normalize_var(lit[1:] if neg else lit)
            term.append((v, neg))
        return [term]

    # Literal suelto
    neg = s.startswith('~')
    v = _normalize_var(s[1:] if neg else s)
    return [[(v, neg)]]


# =============================================================================
# Traductor SOP -> circuito cuántico (oráculo y ^= f(x))
# =============================================================================
def circuit_from_expr(expr, var_order=None):
    """
    Construye un oráculo reversible (x, y) -> (x, y ⊕ f(x)) a partir de 'expr' (PyEDA).
    - var_order: lista de nombres ['x0','x1',...] para fijar el orden de qubits.
    Maneja automáticamente ancillas para MCX con >2 controles.  #FIXEDHERE
    """
    # Orden de variables
    if var_order is None:
        var_order = sorted([_normalize_var(_clean(str(v))) for v in expr.support])
    idx = {name: i for i, name in enumerate(var_order)}

    sop_terms = parse_expr_to_sop(expr)

    # Casos constantes
    if sop_terms == []:
        # y ^= 0
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        return QuantumCircuit(qx, qy, name='oracle')

    if sop_terms == [[]]:
        # y ^= 1
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        qc = QuantumCircuit(qx, qy, name='oracle')
        qc.x(qy[0])
        return qc

    # Nº máximo de controles entre los términos (para reservar ancillas)  #FIXEDHERE
    max_k = max(len(t) for t in sop_terms)
    anc_needed = max(0, max_k - 2)

    qx = QuantumRegister(len(var_order), 'x')
    qy = QuantumRegister(1, 'y')
    qc = QuantumCircuit(qx, qy, name='oracle')

    # Crear registro de ancillas si hace falta  #FIXEDHERE
    anc = None
    if anc_needed > 0:
        anc = QuantumRegister(anc_needed, 'anc')
        qc.add_register(anc)

    # Wrapper MCX que usa ancillas y firma compatible  #FIXEDHERE
    def mcx_auto(ctrls, tgt):
        if anc is None:
            # Por seguridad: para k>2 sin ancillas, probá 'noancilla'
            try:
                qc.mcx(ctrls, tgt)  # Qiskit 2.x puede funcionar si backend lo permite
                return
            except Exception:
                qc.mcx(ctrls, tgt, mode='noancilla')  # deprecado pero aún válido
                return
        # Con ancillas:
        try:
            # Firma Qiskit 2.x (ancilla_qubits=...)  #FIXEDHERE
            qc.mcx(ctrls, tgt, ancilla_qubits=list(anc))
        except TypeError:
            # Firma antigua (mode + ancillas)
            qc.mcx(ctrls, tgt, list(anc), mode='v-chain')

    # Construcción por términos
    for term in sop_terms:
        controls = []
        undo = []
        for (vname, is_neg) in term:
            if vname not in idx:
                raise KeyError(f"Variable '{vname}' no está en var_order {var_order}")
            qi = idx[vname]
            if is_neg:
                qc.x(qx[qi])        # control a 0 -> X antes
                undo.append(qi)
            controls.append(qx[qi])

        k = len(controls)
        if k == 0:
            qc.x(qy[0])            # término constante 1
        elif k == 1:
            qc.cx(controls[0], qy[0])
        elif k == 2:
            qc.ccx(controls[0], controls[1], qy[0])
        else:
            mcx_auto(controls, qy[0])  # usa ancillas si hacen falta  #FIXEDHERE

        for qi in undo:
            qc.x(qx[qi])           # deshacer polaridades

    return qc


# =============================================================================
# 1) Ingreso de tabla de verdad
# =============================================================================
n = int(input("\nIngrese la cantidad de variables (2-4): "))
while n < 2 or n > 4:
    n = int(input("❌ Valor inválido. Ingrese nuevamente (2-4): "))

rows = 2 ** n
X = exprvars('x', n)

print(f"\nVariables: {n}  →  Filas (combinaciones): {rows}")
print("Ingrese las salidas f(x) para cada combinación en orden binario (0..2^n-1)")
print("Use 0, 1 o - (don't care)\n")

# Construir string para PyEDA (p.ej. '01-10-0...')
f_str = ""
for i in range(rows):
    bits = format(i, f"0{n}b")
    while True:
        y = input(f"{i:>2}  ({bits}) → f = ").strip()
        if y in ("0", "1", "-"):
            f_str += y
            break
        else:
            print("⚠️ Ingrese un número válido (0, 1 o -)")

# =============================================================================
# 2) Función booleana y minimización
# =============================================================================
tt = truthtable(X, f_str)
expr_full = truthtable2expr(tt)

print("\nTabla de verdad:\n")
print(tt)

print("\nFunción booleana (no simplificada):")
print(expr_full)

# Minimización (rápida y estable)
try:
    expr_min = expr_full.simplify()
except Exception:
    expr_min = espresso_exprs(expr_full)[0]

print("\nEcuación lógica simplificada:")
print(expr_min)

# =============================================================================
# 3) Traducción a circuito cuántico
# =============================================================================
var_order = [f'x{i}' for i in range(n)]      # formato coherente con el parser
qc = circuit_from_expr(expr_min, var_order=var_order)

print("\nCircuito cuántico equivalente (oráculo y ^= f(x)):")
print(qc.draw(output='text'))

# =============================================================================
# 4) Simulación (opcional)
# =============================================================================
sim = AerSimulator()
_ = transpile(qc, sim)  # verificación de compilación
print("\n✅ Circuito compilado correctamente en AerSimulator.")



PyEDA: 0.28.0
Qiskit: 2.2.2
Python: 3.12.12

Ingrese la cantidad de variables (2-4): 2

Variables: 2  →  Filas (combinaciones): 4
Ingrese las salidas f(x) para cada combinación en orden binario (0..2^n-1)
Use 0, 1 o - (don't care)

 0  (00) → f = 1
 1  (01) → f = 1
 2  (10) → f = 0
 3  (11) → f = 0

Tabla de verdad:

x[1] x[0]
   0    0 : 1
   0    1 : 1
   1    0 : 0
   1    1 : 0


Función booleana (no simplificada):
Or(And(~x[0], ~x[1]), And(x[0], ~x[1]))

Ecuación lógica simplificada:
Or(And(~x[0], ~x[1]), And(x[0], ~x[1]))

Circuito cuántico equivalente (oráculo y ^= f(x)):
     ┌───┐     ┌───┐               
x_0: ┤ X ├──■──┤ X ├───────■───────
     ├───┤  │  ├───┤┌───┐  │  ┌───┐
x_1: ┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├
     └───┘┌─┴─┐└───┘└───┘┌─┴─┐└───┘
  y: ─────┤ X ├──────────┤ X ├─────
          └───┘          └───┘     

✅ Circuito compilado correctamente en AerSimulator.


## Ejemplos con tablas de verdad ya definidas:


**Ejemplo 1: f(x) = 1 X X 0** *(1--0, con "-" para don't care)*

In [ ]:
# -*- coding: utf-8 -*-
"""
Classic2Quantum: Traductor de lógica booleana clásica a lógica cuántica.
Autor: Biancucci, Thiago
Versión: 1.0.0
"""

# =============================================================================
# Instalaciones (para notebook/Colab)
# =============================================================================
!pip install -q qiskit qiskit-aer
!pip install -q pyeda==0.28.0

# =============================================================================
# Imports
# =============================================================================
import os, platform, sys, collections, math
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import AerSimulator

import pyeda
from pyeda.inter import exprvars, truthtable, truthtable2expr, espresso_exprs
# Fix para Python 3.10+ (compatibilidad pyeda 0.28)
import collections.abc
collections.Sequence = collections.abc.Sequence


# =============================================================================
# Utilidades
# =============================================================================
def clear_console():
    """Limpia consola o salida embebida."""
    if 'ipykernel' in sys.modules:  # Jupyter / Colab
        from IPython.display import clear_output
        clear_output(wait=True)
    else:
        os.system('cls' if platform.system() == 'Windows' else 'clear')

clear_console()

print("PyEDA:", pyeda.__version__)
print("Qiskit:", qiskit.__version__)
print("Python:", sys.version.split()[0])


# ---------- helpers de parser ----------
def _clean(s: str) -> str:
    return s.replace(' ', '')

def _normalize_var(token: str) -> str:
    # 'x[0]' -> 'x0'
    if token.startswith('x[') and token.endswith(']'):
        return 'x' + token[2:-1]
    return token

def _split_top_level(csv: str) -> list:
    """Separa por comas a nivel de paréntesis 0 (no rompe subexpresiones)."""
    items, depth, start = [], 0, 0
    for i, ch in enumerate(csv):
        if ch == '(':
            depth += 1
        elif ch == ')':
            depth -= 1
        elif ch == ',' and depth == 0:
            items.append(csv[start:i])
            start = i + 1
    items.append(csv[start:])
    return items

def _unwrap(op: str, s: str) -> str:
    """
    Devuelve el contenido dentro de op(...).
    Ej: _unwrap('Or', 'Or(A,B)') -> 'A,B'
    """
    s = _clean(s)
    assert s.startswith(op + '(') and s.endswith(')'), f"Formato inesperado: {s}"
    return s[len(op) + 1 : -1]   # <- correcto (antes se comía letras)


def parse_expr_to_sop(expr) -> list:
    """
    Devuelve SOP como lista de términos:
      [[('x0', True), ('x1', False)], [('x0', False)], ...]
    Soporta: Or(...), And(...), literales (~x[1], x0,...)
    """
    s = _clean(str(expr))

    if s == '0':
        return []      # y ^= 0
    if s == '1':
        return [[]]    # y ^= 1

    # Or(...)
    if s.startswith('Or(') and s.endswith(')'):
        inner = _unwrap('Or', s)
        parts = _split_top_level(inner)
        terms = []
        for p in parts:
            terms.extend(parse_expr_to_sop(p))
        return terms

    # And(...)
    if s.startswith('And(') and s.endswith(')'):
        inner = _unwrap('And', s)
        lits = _split_top_level(inner)
        term = []
        for lit in lits:
            lit = _clean(lit)
            neg = lit.startswith('~')
            v = _normalize_var(lit[1:] if neg else lit)
            term.append((v, neg))
        return [term]

    # Literal suelto
    neg = s.startswith('~')
    v = _normalize_var(s[1:] if neg else s)
    return [[(v, neg)]]


# =============================================================================
# Traductor SOP -> circuito cuántico (oráculo y ^= f(x))
# =============================================================================
def circuit_from_expr(expr, var_order=None):
    """
    Construye un oráculo reversible (x, y) -> (x, y ⊕ f(x)) a partir de 'expr' (PyEDA).
    - var_order: lista de nombres ['x0','x1',...] para fijar el orden de qubits.
    Maneja automáticamente ancillas para MCX con >2 controles.  #FIXEDHERE
    """
    # Orden de variables
    if var_order is None:
        var_order = sorted([_normalize_var(_clean(str(v))) for v in expr.support])
    idx = {name: i for i, name in enumerate(var_order)}

    sop_terms = parse_expr_to_sop(expr)

    # Casos constantes
    if sop_terms == []:
        # y ^= 0
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        return QuantumCircuit(qx, qy, name='oracle')

    if sop_terms == [[]]:
        # y ^= 1
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        qc = QuantumCircuit(qx, qy, name='oracle')
        qc.x(qy[0])
        return qc

    # Nº máximo de controles entre los términos (para reservar ancillas)  #FIXEDHERE
    max_k = max(len(t) for t in sop_terms)
    anc_needed = max(0, max_k - 2)

    qx = QuantumRegister(len(var_order), 'x')
    qy = QuantumRegister(1, 'y')
    qc = QuantumCircuit(qx, qy, name='oracle')

    # Crear registro de ancillas si hace falta  #FIXEDHERE
    anc = None
    if anc_needed > 0:
        anc = QuantumRegister(anc_needed, 'anc')
        qc.add_register(anc)

    # Wrapper MCX que usa ancillas y firma compatible  #FIXEDHERE
    def mcx_auto(ctrls, tgt):
        if anc is None:
            # Por seguridad: para k>2 sin ancillas, probá 'noancilla'
            try:
                qc.mcx(ctrls, tgt)  # Qiskit 2.x puede funcionar si backend lo permite
                return
            except Exception:
                qc.mcx(ctrls, tgt, mode='noancilla')  # deprecado pero aún válido
                return
        # Con ancillas:
        try:
            # Firma Qiskit 2.x (ancilla_qubits=...)  #FIXEDHERE
            qc.mcx(ctrls, tgt, ancilla_qubits=list(anc))
        except TypeError:
            # Firma antigua (mode + ancillas)
            qc.mcx(ctrls, tgt, list(anc), mode='v-chain')

    # Construcción por términos
    for term in sop_terms:
        controls = []
        undo = []
        for (vname, is_neg) in term:
            if vname not in idx:
                raise KeyError(f"Variable '{vname}' no está en var_order {var_order}")
            qi = idx[vname]
            if is_neg:
                qc.x(qx[qi])        # control a 0 -> X antes
                undo.append(qi)
            controls.append(qx[qi])

        k = len(controls)
        if k == 0:
            qc.x(qy[0])            # término constante 1
        elif k == 1:
            qc.cx(controls[0], qy[0])
        elif k == 2:
            qc.ccx(controls[0], controls[1], qy[0])
        else:
            mcx_auto(controls, qy[0])  # usa ancillas si hacen falta  #FIXEDHERE

        for qi in undo:
            qc.x(qx[qi])           # deshacer polaridades

    return qc


# =============================================================================
# 1) Ingreso de tabla de verdad (ejemplo)
# =============================================================================
# Cambiá esto por tu TT; longitud debe ser 2^n
ex2 = "1--0"        # 4 variables, 16 entradas
n = int(math.log(len(ex2), 2))
rows = 2 ** n
X = exprvars('x', n)

print(f"\nVariables: {n}  →  Filas (combinaciones): {rows}")

f_str = ex2

# =============================================================================
# 2) Función booleana y minimización
# =============================================================================
tt = truthtable(X, f_str)
expr_full = truthtable2expr(tt)

print("\nTabla de verdad:\n")
print(tt)

print("\nFunción booleana (no simplificada):")
print(expr_full)

# Minimización (rápida y estable)
try:
    expr_min = expr_full.simplify()
except Exception:
    expr_min = espresso_exprs(expr_full)[0]

print("\nEcuación lógica simplificada:")
print(expr_min)

# =============================================================================
# 3) Traducción a circuito cuántico
# =============================================================================
var_order = [f'x{i}' for i in range(n)]      # formato coherente con el parser
qc = circuit_from_expr(expr_min, var_order=var_order)

print("\nCircuito cuántico equivalente (oráculo y ^= f(x)):")
print(qc.draw(output='text'))

# =============================================================================
# 4) Simulación (opcional)
# =============================================================================
sim = AerSimulator()
_ = transpile(qc, sim)  # verificación de compilación
print("\n✅ Circuito compilado correctamente en AerSimulator.")


PyEDA: 0.28.0
Qiskit: 2.2.2
Python: 3.12.12

Variables: 2  →  Filas (combinaciones): 4

Tabla de verdad:

x[1] x[0]
   0    0 : 1
   0    1 : -
   1    0 : -
   1    1 : 0


Función booleana (no simplificada):
And(~x[0], ~x[1])

Ecuación lógica simplificada:
And(~x[0], ~x[1])

Circuito cuántico equivalente (oráculo y ^= f(x)):
     ┌───┐     ┌───┐
x_0: ┤ X ├──■──┤ X ├
     ├───┤  │  ├───┤
x_1: ┤ X ├──■──┤ X ├
     └───┘┌─┴─┐└───┘
  y: ─────┤ X ├─────
          └───┘     

✅ Circuito compilado correctamente en AerSimulator.


Ejemplo 2, 3 y 4

In [ ]:
# -*- coding: utf-8 -*-
"""
Classic2Quantum: Traductor de lógica booleana clásica a lógica cuántica.
Autor: Biancucci, Thiago
Versión: 1.0.0
"""

# =============================================================================
# Instalaciones (para notebook/Colab)
# =============================================================================
!pip install -q qiskit qiskit-aer
!pip install -q pyeda==0.28.0

# =============================================================================
# Imports
# =============================================================================
import os, platform, sys, collections, math
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import AerSimulator

import pyeda
from pyeda.inter import exprvars, truthtable, truthtable2expr, espresso_exprs
# Fix para Python 3.10+ (compatibilidad pyeda 0.28)
import collections.abc
collections.Sequence = collections.abc.Sequence


# =============================================================================
# Utilidades
# =============================================================================
def clear_console():
    """Limpia consola o salida embebida."""
    if 'ipykernel' in sys.modules:  # Jupyter / Colab
        from IPython.display import clear_output
        clear_output(wait=True)
    else:
        os.system('cls' if platform.system() == 'Windows' else 'clear')

clear_console()

print("PyEDA:", pyeda.__version__)
print("Qiskit:", qiskit.__version__)
print("Python:", sys.version.split()[0])


# ---------- helpers de parser ----------
def _clean(s: str) -> str:
    return s.replace(' ', '')

def _normalize_var(token: str) -> str:
    # 'x[0]' -> 'x0'
    if token.startswith('x[') and token.endswith(']'):
        return 'x' + token[2:-1]
    return token

def _split_top_level(csv: str) -> list:
    """Separa por comas a nivel de paréntesis 0 (no rompe subexpresiones)."""
    items, depth, start = [], 0, 0
    for i, ch in enumerate(csv):
        if ch == '(':
            depth += 1
        elif ch == ')':
            depth -= 1
        elif ch == ',' and depth == 0:
            items.append(csv[start:i])
            start = i + 1
    items.append(csv[start:])
    return items

def _unwrap(op: str, s: str) -> str:
    """
    Devuelve el contenido dentro de op(...).
    Ej: _unwrap('Or', 'Or(A,B)') -> 'A,B'
    """
    s = _clean(s)
    assert s.startswith(op + '(') and s.endswith(')'), f"Formato inesperado: {s}"
    return s[len(op) + 1 : -1]   # <- correcto (antes se comía letras)


def parse_expr_to_sop(expr) -> list:
    """
    Devuelve SOP como lista de términos:
      [[('x0', True), ('x1', False)], [('x0', False)], ...]
    Soporta: Or(...), And(...), literales (~x[1], x0,...)
    """
    s = _clean(str(expr))

    if s == '0':
        return []      # y ^= 0
    if s == '1':
        return [[]]    # y ^= 1

    # Or(...)
    if s.startswith('Or(') and s.endswith(')'):
        inner = _unwrap('Or', s)
        parts = _split_top_level(inner)
        terms = []
        for p in parts:
            terms.extend(parse_expr_to_sop(p))
        return terms

    # And(...)
    if s.startswith('And(') and s.endswith(')'):
        inner = _unwrap('And', s)
        lits = _split_top_level(inner)
        term = []
        for lit in lits:
            lit = _clean(lit)
            neg = lit.startswith('~')
            v = _normalize_var(lit[1:] if neg else lit)
            term.append((v, neg))
        return [term]

    # Literal suelto
    neg = s.startswith('~')
    v = _normalize_var(s[1:] if neg else s)
    return [[(v, neg)]]


# =============================================================================
# Traductor SOP -> circuito cuántico (oráculo y ^= f(x))
# =============================================================================
def circuit_from_expr(expr, var_order=None):
    """
    Construye un oráculo reversible (x, y) -> (x, y ⊕ f(x)) a partir de 'expr' (PyEDA).
    - var_order: lista de nombres ['x0','x1',...] para fijar el orden de qubits.
    Maneja automáticamente ancillas para MCX con >2 controles.  #FIXEDHERE
    """
    # Orden de variables
    if var_order is None:
        var_order = sorted([_normalize_var(_clean(str(v))) for v in expr.support])
    idx = {name: i for i, name in enumerate(var_order)}

    sop_terms = parse_expr_to_sop(expr)

    # Casos constantes
    if sop_terms == []:
        # y ^= 0
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        return QuantumCircuit(qx, qy, name='oracle')

    if sop_terms == [[]]:
        # y ^= 1
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        qc = QuantumCircuit(qx, qy, name='oracle')
        qc.x(qy[0])
        return qc

    # Nº máximo de controles entre los términos (para reservar ancillas)  #FIXEDHERE
    max_k = max(len(t) for t in sop_terms)
    anc_needed = max(0, max_k - 2)

    qx = QuantumRegister(len(var_order), 'x')
    qy = QuantumRegister(1, 'y')
    qc = QuantumCircuit(qx, qy, name='oracle')

    # Crear registro de ancillas si hace falta  #FIXEDHERE
    anc = None
    if anc_needed > 0:
        anc = QuantumRegister(anc_needed, 'anc')
        qc.add_register(anc)

    # Wrapper MCX que usa ancillas y firma compatible  #FIXEDHERE
    def mcx_auto(ctrls, tgt):
        if anc is None:
            # Por seguridad: para k>2 sin ancillas, probá 'noancilla'
            try:
                qc.mcx(ctrls, tgt)  # Qiskit 2.x puede funcionar si backend lo permite
                return
            except Exception:
                qc.mcx(ctrls, tgt, mode='noancilla')  # deprecado pero aún válido
                return
        # Con ancillas:
        try:
            # Firma Qiskit 2.x (ancilla_qubits=...)  #FIXEDHERE
            qc.mcx(ctrls, tgt, ancilla_qubits=list(anc))
        except TypeError:
            # Firma antigua (mode + ancillas)
            qc.mcx(ctrls, tgt, list(anc), mode='v-chain')

    # Construcción por términos
    for term in sop_terms:
        controls = []
        undo = []
        for (vname, is_neg) in term:
            if vname not in idx:
                raise KeyError(f"Variable '{vname}' no está en var_order {var_order}")
            qi = idx[vname]
            if is_neg:
                qc.x(qx[qi])        # control a 0 -> X antes
                undo.append(qi)
            controls.append(qx[qi])

        k = len(controls)
        if k == 0:
            qc.x(qy[0])            # término constante 1
        elif k == 1:
            qc.cx(controls[0], qy[0])
        elif k == 2:
            qc.ccx(controls[0], controls[1], qy[0])
        else:
            mcx_auto(controls, qy[0])  # usa ancillas si hacen falta  #FIXEDHERE

        for qi in undo:
            qc.x(qx[qi])           # deshacer polaridades

    return qc


# =============================================================================
# 1) Ingreso de tabla de verdad (ejemplo)
# =============================================================================
# Cambiá esto por tu TT; longitud debe ser 2^n
ex2 = "11110000"        # 4 variables, 16 entradas
n = int(math.log(len(ex2), 2))
rows = 2 ** n
X = exprvars('x', n)

print(f"\nVariables: {n}  →  Filas (combinaciones): {rows}")

f_str = ex2

# =============================================================================
# 2) Función booleana y minimización
# =============================================================================
tt = truthtable(X, f_str)
expr_full = truthtable2expr(tt)

print("\nTabla de verdad:\n")
print(tt)

print("\nFunción booleana (no simplificada):")
print(expr_full)

# Minimización (rápida y estable)
try:
    expr_min = expr_full.simplify()
except Exception:
    expr_min = espresso_exprs(expr_full)[0]

print("\nEcuación lógica simplificada:")
print(expr_min)

# =============================================================================
# 3) Traducción a circuito cuántico
# =============================================================================
var_order = [f'x{i}' for i in range(n)]      # formato coherente con el parser
qc = circuit_from_expr(expr_min, var_order=var_order)

print("\nCircuito cuántico equivalente (oráculo y ^= f(x)):")
print(qc.draw(output='text'))

# =============================================================================
# 4) Simulación (opcional)
# =============================================================================
sim = AerSimulator()
_ = transpile(qc, sim)  # verificación de compilación
print("\n✅ Circuito compilado correctamente en AerSimulator.")


In [ ]:
# -*- coding: utf-8 -*-
"""
Classic2Quantum: Traductor de lógica booleana clásica a lógica cuántica.
Autor: Biancucci, Thiago
Versión: 1.0.0
"""

# =============================================================================
# Instalaciones (para notebook/Colab)
# =============================================================================
!pip install -q qiskit qiskit-aer
!pip install -q pyeda==0.28.0

# =============================================================================
# Imports
# =============================================================================
import os, platform, sys, collections, math
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import AerSimulator

import pyeda
from pyeda.inter import exprvars, truthtable, truthtable2expr, espresso_exprs
# Fix para Python 3.10+ (compatibilidad pyeda 0.28)
import collections.abc
collections.Sequence = collections.abc.Sequence


# =============================================================================
# Utilidades
# =============================================================================
def clear_console():
    """Limpia consola o salida embebida."""
    if 'ipykernel' in sys.modules:  # Jupyter / Colab
        from IPython.display import clear_output
        clear_output(wait=True)
    else:
        os.system('cls' if platform.system() == 'Windows' else 'clear')

clear_console()

print("PyEDA:", pyeda.__version__)
print("Qiskit:", qiskit.__version__)
print("Python:", sys.version.split()[0])


# ---------- helpers de parser ----------
def _clean(s: str) -> str:
    return s.replace(' ', '')

def _normalize_var(token: str) -> str:
    # 'x[0]' -> 'x0'
    if token.startswith('x[') and token.endswith(']'):
        return 'x' + token[2:-1]
    return token

def _split_top_level(csv: str) -> list:
    """Separa por comas a nivel de paréntesis 0 (no rompe subexpresiones)."""
    items, depth, start = [], 0, 0
    for i, ch in enumerate(csv):
        if ch == '(':
            depth += 1
        elif ch == ')':
            depth -= 1
        elif ch == ',' and depth == 0:
            items.append(csv[start:i])
            start = i + 1
    items.append(csv[start:])
    return items

def _unwrap(op: str, s: str) -> str:
    """
    Devuelve el contenido dentro de op(...).
    Ej: _unwrap('Or', 'Or(A,B)') -> 'A,B'
    """
    s = _clean(s)
    assert s.startswith(op + '(') and s.endswith(')'), f"Formato inesperado: {s}"
    return s[len(op) + 1 : -1]   # <- correcto (antes se comía letras)


def parse_expr_to_sop(expr) -> list:
    """
    Devuelve SOP como lista de términos:
      [[('x0', True), ('x1', False)], [('x0', False)], ...]
    Soporta: Or(...), And(...), literales (~x[1], x0,...)
    """
    s = _clean(str(expr))

    if s == '0':
        return []      # y ^= 0
    if s == '1':
        return [[]]    # y ^= 1

    # Or(...)
    if s.startswith('Or(') and s.endswith(')'):
        inner = _unwrap('Or', s)
        parts = _split_top_level(inner)
        terms = []
        for p in parts:
            terms.extend(parse_expr_to_sop(p))
        return terms

    # And(...)
    if s.startswith('And(') and s.endswith(')'):
        inner = _unwrap('And', s)
        lits = _split_top_level(inner)
        term = []
        for lit in lits:
            lit = _clean(lit)
            neg = lit.startswith('~')
            v = _normalize_var(lit[1:] if neg else lit)
            term.append((v, neg))
        return [term]

    # Literal suelto
    neg = s.startswith('~')
    v = _normalize_var(s[1:] if neg else s)
    return [[(v, neg)]]


# =============================================================================
# Traductor SOP -> circuito cuántico (oráculo y ^= f(x))
# =============================================================================
def circuit_from_expr(expr, var_order=None):
    """
    Construye un oráculo reversible (x, y) -> (x, y ⊕ f(x)) a partir de 'expr' (PyEDA).
    - var_order: lista de nombres ['x0','x1',...] para fijar el orden de qubits.
    Maneja automáticamente ancillas para MCX con >2 controles.  #FIXEDHERE
    """
    # Orden de variables
    if var_order is None:
        var_order = sorted([_normalize_var(_clean(str(v))) for v in expr.support])
    idx = {name: i for i, name in enumerate(var_order)}

    sop_terms = parse_expr_to_sop(expr)

    # Casos constantes
    if sop_terms == []:
        # y ^= 0
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        return QuantumCircuit(qx, qy, name='oracle')

    if sop_terms == [[]]:
        # y ^= 1
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        qc = QuantumCircuit(qx, qy, name='oracle')
        qc.x(qy[0])
        return qc

    # Nº máximo de controles entre los términos (para reservar ancillas)  #FIXEDHERE
    max_k = max(len(t) for t in sop_terms)
    anc_needed = max(0, max_k - 2)

    qx = QuantumRegister(len(var_order), 'x')
    qy = QuantumRegister(1, 'y')
    qc = QuantumCircuit(qx, qy, name='oracle')

    # Crear registro de ancillas si hace falta  #FIXEDHERE
    anc = None
    if anc_needed > 0:
        anc = QuantumRegister(anc_needed, 'anc')
        qc.add_register(anc)

    # Wrapper MCX que usa ancillas y firma compatible  #FIXEDHERE
    def mcx_auto(ctrls, tgt):
        if anc is None:
            # Por seguridad: para k>2 sin ancillas, probá 'noancilla'
            try:
                qc.mcx(ctrls, tgt)  # Qiskit 2.x puede funcionar si backend lo permite
                return
            except Exception:
                qc.mcx(ctrls, tgt, mode='noancilla')  # deprecado pero aún válido
                return
        # Con ancillas:
        try:
            # Firma Qiskit 2.x (ancilla_qubits=...)  #FIXEDHERE
            qc.mcx(ctrls, tgt, ancilla_qubits=list(anc))
        except TypeError:
            # Firma antigua (mode + ancillas)
            qc.mcx(ctrls, tgt, list(anc), mode='v-chain')

    # Construcción por términos
    for term in sop_terms:
        controls = []
        undo = []
        for (vname, is_neg) in term:
            if vname not in idx:
                raise KeyError(f"Variable '{vname}' no está en var_order {var_order}")
            qi = idx[vname]
            if is_neg:
                qc.x(qx[qi])        # control a 0 -> X antes
                undo.append(qi)
            controls.append(qx[qi])

        k = len(controls)
        if k == 0:
            qc.x(qy[0])            # término constante 1
        elif k == 1:
            qc.cx(controls[0], qy[0])
        elif k == 2:
            qc.ccx(controls[0], controls[1], qy[0])
        else:
            mcx_auto(controls, qy[0])  # usa ancillas si hacen falta  #FIXEDHERE

        for qi in undo:
            qc.x(qx[qi])           # deshacer polaridades

    return qc


# =============================================================================
# 1) Ingreso de tabla de verdad (ejemplo)
# =============================================================================
# Cambiá esto por tu TT; longitud debe ser 2^n
ex3 = "11--0000"        # 4 variables, 16 entradas
n = int(math.log(len(ex3), 2))
rows = 2 ** n
X = exprvars('x', n)

print(f"\nVariables: {n}  →  Filas (combinaciones): {rows}")

f_str = ex3

# =============================================================================
# 2) Función booleana y minimización
# =============================================================================
tt = truthtable(X, f_str)
expr_full = truthtable2expr(tt)

print("\nTabla de verdad:\n")
print(tt)

print("\nFunción booleana (no simplificada):")
print(expr_full)

# Minimización (rápida y estable)
try:
    expr_min = expr_full.simplify()
except Exception:
    expr_min = espresso_exprs(expr_full)[0]

print("\nEcuación lógica simplificada:")
print(expr_min)

# =============================================================================
# 3) Traducción a circuito cuántico
# =============================================================================
var_order = [f'x{i}' for i in range(n)]      # formato coherente con el parser
qc = circuit_from_expr(expr_min, var_order=var_order)

print("\nCircuito cuántico equivalente (oráculo y ^= f(x)):")
print(qc.draw(output='text'))

# =============================================================================
# 4) Simulación (opcional)
# =============================================================================
sim = AerSimulator()
_ = transpile(qc, sim)  # verificación de compilación
print("\n✅ Circuito compilado correctamente en AerSimulator.")


PyEDA: 0.28.0
Qiskit: 2.2.2
Python: 3.12.12

Variables: 3  →  Filas (combinaciones): 8

Tabla de verdad:

x[2] x[1] x[0]
   0    0    0 : 1
   0    0    1 : 1
   0    1    0 : -
   0    1    1 : -
   1    0    0 : 0
   1    0    1 : 0
   1    1    0 : 0
   1    1    1 : 0


Función booleana (no simplificada):
Or(And(~x[0], ~x[1], ~x[2]), And(x[0], ~x[1], ~x[2]))

Ecuación lógica simplificada:
Or(And(~x[0], ~x[1], ~x[2]), And(x[0], ~x[1], ~x[2]))

Circuito cuántico equivalente (oráculo y ^= f(x)):
     ┌───┐     ┌───┐               
x_0: ┤ X ├──■──┤ X ├───────■───────
     ├───┤  │  ├───┤┌───┐  │  ┌───┐
x_1: ┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├
     ├───┤  │  ├───┤├───┤  │  ├───┤
x_2: ┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├
     └───┘┌─┴─┐└───┘└───┘┌─┴─┐└───┘
  y: ─────┤ X ├──────────┤ X ├─────
          └───┘          └───┘     
anc: ──────────────────────────────
                                   

✅ Circuito compilado correctamente en AerSimulator.


In [ ]:
# -*- coding: utf-8 -*-
"""
Classic2Quantum: Traductor de lógica booleana clásica a lógica cuántica.
Autor: Biancucci, Thiago
Versión: 1.0.0
"""

# =============================================================================
# Instalaciones (para notebook/Colab)
# =============================================================================
!pip install -q qiskit qiskit-aer
!pip install -q pyeda==0.28.0

# =============================================================================
# Imports
# =============================================================================
import os, platform, sys, collections, math
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import AerSimulator

import pyeda
from pyeda.inter import exprvars, truthtable, truthtable2expr, espresso_exprs
# Fix para Python 3.10+ (compatibilidad pyeda 0.28)
import collections.abc
collections.Sequence = collections.abc.Sequence


# =============================================================================
# Utilidades
# =============================================================================
def clear_console():
    """Limpia consola o salida embebida."""
    if 'ipykernel' in sys.modules:  # Jupyter / Colab
        from IPython.display import clear_output
        clear_output(wait=True)
    else:
        os.system('cls' if platform.system() == 'Windows' else 'clear')

clear_console()

print("PyEDA:", pyeda.__version__)
print("Qiskit:", qiskit.__version__)
print("Python:", sys.version.split()[0])


# ---------- helpers de parser ----------
def _clean(s: str) -> str:
    return s.replace(' ', '')

def _normalize_var(token: str) -> str:
    # 'x[0]' -> 'x0'
    if token.startswith('x[') and token.endswith(']'):
        return 'x' + token[2:-1]
    return token

def _split_top_level(csv: str) -> list:
    """Separa por comas a nivel de paréntesis 0 (no rompe subexpresiones)."""
    items, depth, start = [], 0, 0
    for i, ch in enumerate(csv):
        if ch == '(':
            depth += 1
        elif ch == ')':
            depth -= 1
        elif ch == ',' and depth == 0:
            items.append(csv[start:i])
            start = i + 1
    items.append(csv[start:])
    return items

def _unwrap(op: str, s: str) -> str:
    """
    Devuelve el contenido dentro de op(...).
    Ej: _unwrap('Or', 'Or(A,B)') -> 'A,B'
    """
    s = _clean(s)
    assert s.startswith(op + '(') and s.endswith(')'), f"Formato inesperado: {s}"
    return s[len(op) + 1 : -1]   # <- correcto (antes se comía letras)


def parse_expr_to_sop(expr) -> list:
    """
    Devuelve SOP como lista de términos:
      [[('x0', True), ('x1', False)], [('x0', False)], ...]
    Soporta: Or(...), And(...), literales (~x[1], x0,...)
    """
    s = _clean(str(expr))

    if s == '0':
        return []      # y ^= 0
    if s == '1':
        return [[]]    # y ^= 1

    # Or(...)
    if s.startswith('Or(') and s.endswith(')'):
        inner = _unwrap('Or', s)
        parts = _split_top_level(inner)
        terms = []
        for p in parts:
            terms.extend(parse_expr_to_sop(p))
        return terms

    # And(...)
    if s.startswith('And(') and s.endswith(')'):
        inner = _unwrap('And', s)
        lits = _split_top_level(inner)
        term = []
        for lit in lits:
            lit = _clean(lit)
            neg = lit.startswith('~')
            v = _normalize_var(lit[1:] if neg else lit)
            term.append((v, neg))
        return [term]

    # Literal suelto
    neg = s.startswith('~')
    v = _normalize_var(s[1:] if neg else s)
    return [[(v, neg)]]


# =============================================================================
# Traductor SOP -> circuito cuántico (oráculo y ^= f(x))
# =============================================================================
def circuit_from_expr(expr, var_order=None):
    """
    Construye un oráculo reversible (x, y) -> (x, y ⊕ f(x)) a partir de 'expr' (PyEDA).
    - var_order: lista de nombres ['x0','x1',...] para fijar el orden de qubits.
    Maneja automáticamente ancillas para MCX con >2 controles.  #FIXEDHERE
    """
    # Orden de variables
    if var_order is None:
        var_order = sorted([_normalize_var(_clean(str(v))) for v in expr.support])
    idx = {name: i for i, name in enumerate(var_order)}

    sop_terms = parse_expr_to_sop(expr)

    # Casos constantes
    if sop_terms == []:
        # y ^= 0
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        return QuantumCircuit(qx, qy, name='oracle')

    if sop_terms == [[]]:
        # y ^= 1
        qx = QuantumRegister(len(var_order), 'x')
        qy = QuantumRegister(1, 'y')
        qc = QuantumCircuit(qx, qy, name='oracle')
        qc.x(qy[0])
        return qc

    # Nº máximo de controles entre los términos (para reservar ancillas)  #FIXEDHERE
    max_k = max(len(t) for t in sop_terms)
    anc_needed = max(0, max_k - 2)

    qx = QuantumRegister(len(var_order), 'x')
    qy = QuantumRegister(1, 'y')
    qc = QuantumCircuit(qx, qy, name='oracle')

    # Crear registro de ancillas si hace falta  #FIXEDHERE
    anc = None
    if anc_needed > 0:
        anc = QuantumRegister(anc_needed, 'anc')
        qc.add_register(anc)

    # Wrapper MCX que usa ancillas y firma compatible  #FIXEDHERE
    def mcx_auto(ctrls, tgt):
        if anc is None:
            # Por seguridad: para k>2 sin ancillas, probá 'noancilla'
            try:
                qc.mcx(ctrls, tgt)  # Qiskit 2.x puede funcionar si backend lo permite
                return
            except Exception:
                qc.mcx(ctrls, tgt, mode='noancilla')  # deprecado pero aún válido
                return
        # Con ancillas:
        try:
            # Firma Qiskit 2.x (ancilla_qubits=...)  #FIXEDHERE
            qc.mcx(ctrls, tgt, ancilla_qubits=list(anc))
        except TypeError:
            # Firma antigua (mode + ancillas)
            qc.mcx(ctrls, tgt, list(anc), mode='v-chain')

    # Construcción por términos
    for term in sop_terms:
        controls = []
        undo = []
        for (vname, is_neg) in term:
            if vname not in idx:
                raise KeyError(f"Variable '{vname}' no está en var_order {var_order}")
            qi = idx[vname]
            if is_neg:
                qc.x(qx[qi])        # control a 0 -> X antes
                undo.append(qi)
            controls.append(qx[qi])

        k = len(controls)
        if k == 0:
            qc.x(qy[0])            # término constante 1
        elif k == 1:
            qc.cx(controls[0], qy[0])
        elif k == 2:
            qc.ccx(controls[0], controls[1], qy[0])
        else:
            mcx_auto(controls, qy[0])  # usa ancillas si hacen falta  #FIXEDHERE

        for qi in undo:
            qc.x(qx[qi])           # deshacer polaridades

    return qc


# =============================================================================
# 1) Ingreso de tabla de verdad (ejemplo)
# =============================================================================
# Cambiá esto por tu TT; longitud debe ser 2^n
ex4 = "1111000010100011"        # 4 variables, 16 entradas
n = int(math.log(len(ex4), 2))
rows = 2 ** n
X = exprvars('x', n)

print(f"\nVariables: {n}  →  Filas (combinaciones): {rows}")

f_str = ex4

# =============================================================================
# 2) Función booleana y minimización
# =============================================================================
tt = truthtable(X, f_str)
expr_full = truthtable2expr(tt)

print("\nTabla de verdad:\n")
print(tt)

print("\nFunción booleana (no simplificada):")
print(expr_full)

# Minimización (rápida y estable)
try:
    expr_min = expr_full.simplify()
except Exception:
    expr_min = espresso_exprs(expr_full)[0]

print("\nEcuación lógica simplificada:")
print(expr_min)

# =============================================================================
# 3) Traducción a circuito cuántico
# =============================================================================
var_order = [f'x{i}' for i in range(n)]      # formato coherente con el parser
qc = circuit_from_expr(expr_min, var_order=var_order)

print("\nCircuito cuántico equivalente (oráculo y ^= f(x)):")
print(qc.draw(output='text'))

# =============================================================================
# 4) Simulación (opcional)
# =============================================================================
sim = AerSimulator()
_ = transpile(qc, sim)  # verificación de compilación
print("\n✅ Circuito compilado correctamente en AerSimulator.")


PyEDA: 0.28.0
Qiskit: 2.2.2
Python: 3.12.12

Variables: 4  →  Filas (combinaciones): 16

Tabla de verdad:

x[3] x[2] x[1] x[0]
   0    0    0    0 : 1
   0    0    0    1 : 1
   0    0    1    0 : 1
   0    0    1    1 : 1
   0    1    0    0 : 0
   0    1    0    1 : 0
   0    1    1    0 : 0
   0    1    1    1 : 0
   1    0    0    0 : 1
   1    0    0    1 : 0
   1    0    1    0 : 1
   1    0    1    1 : 0
   1    1    0    0 : 0
   1    1    0    1 : 0
   1    1    1    0 : 1
   1    1    1    1 : 1


Función booleana (no simplificada):
Or(And(~x[0], ~x[1], ~x[2], ~x[3]), And(x[0], ~x[1], ~x[2], ~x[3]), And(~x[0], x[1], ~x[2], ~x[3]), And(x[0], x[1], ~x[2], ~x[3]), And(~x[0], ~x[1], ~x[2], x[3]), And(~x[0], x[1], ~x[2], x[3]), And(~x[0], x[1], x[2], x[3]), And(x[0], x[1], x[2], x[3]))

Ecuación lógica simplificada:
Or(And(~x[0], ~x[1], ~x[2], ~x[3]), And(x[0], ~x[1], ~x[2], ~x[3]), And(~x[0], x[1], ~x[2], ~x[3]), And(x[0], x[1], ~x[2], ~x[3]), And(~x[0], ~x[1], ~x[2], x[3]), And(

# Patch Notes

v.0.8.7
-

Classic2Quantum (0.8.7) se encuentra funcionando y ejecutandose en las versiones de PyEDA: 0.28.0; Qiskit: 2.2.1 ; Python: 3.12.11, a la fecha 07/10/25.

v.0.8.9
-

Cambios clave:
- MCX (>2 controles): ancillas automáticas según k-2.  #FIXEDHERE
- Qiskit 2.x: wrapper mcx_auto → usa mcx(ctrls, tgt, ancilla_qubits=anc).  #FIXEDHERE
- Retrocompat: si la firma antigua es requerida, mcx(ctrls, tgt, anc, mode='v-chain').  #FIXEDHERE
- Eliminado fallback obsoleto: mct ya no se usa.  #FIXEDHERE
- Parser Or/And: _unwrap corregido (no corta letras).  #FIXEDHERE
- Se corrigen los errores: “No ancillas provided …” y “QuantumCircuit has no attribute 'mct'”.
- Traducción estable para términos con muchos literales (MCX de alta aridad).
- Compatible con Qiskit ≥ 2.x y expresiones de PyEDA.

Impacto:
- Los circuitos pueden incluir un registro de ancillas cuando sea necesario (transparente).

v.0.9.0
-

Cambios clave:
- Los cambios en los ejemplos de la v.0.8.9 fueron trasladados a la funcionalidad principal.

Impacto:
- Ahora es posible cargar tablas de verdad con más de dos variables y obtener un resultado positivo.

v.1.0.0
-

- Versión estable, se pueden ejecutar de 2 a 4 variables (se realizaron testeos manuales)

Roadmap para futuras versiones:
-

- Verificar el correcto funcionamiento del sistema y las simplificaciones; se deberían plantear más circuitos ya conocidos, simplificados y corroborar que coincidan con los resultados de Classic2Quantum con una tasa de coincidencia mayor al 95%.
- Mejorar funcionalidades y acelerar procesamiento. Se busca bajar los tiempos de carga un 30% mediante el uso de librerías más livianas y arquitectura más eficiente.
- Corregir posibles errores no detectados.

